# Load images stored

In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50
import tensorflow as tf
from keras import losses
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def load_images(train, directory):
    
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 224
    train_image = []
    for name in train['image_id']:
        path = directory + 'images/' + name + '.jpg'
        img = cv2.imread(path)
        image = cv2.resize(img, (img_size, img_size))
        train_image.append(image)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [3]:
def load_images_augmented(directory):
    
    # This function loads the augmented images and the augmented csv file
    df_train = pd.read_csv(directory + 'augmented.csv')
    
    train_image = []
    for name in df_train['image_id']:
        path = directory + 'images_resized_augmented/' + name + '.jpg'
        img = cv2.imread(path)
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return  train_image_array, df_train

In [4]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
x_train, df_train = load_images_augmented(directory)
df_test = pd.read_csv(directory + 'test.csv')
x_test = load_images(df_test, directory)


# Split training set into train and validation

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y_train = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].to_numpy()

x_train_original, y_train_original = shuffle(x_train, y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train_original, y_train_original, test_size = 0.2)

print('Size of x_train: ', x_train.shape)
print('Size of x_val: ', x_val.shape)

Size of x_train:  (7284, 224, 224, 3)
Size of x_val:  (1821, 224, 224, 3)


In [17]:
def predict_and_save(model, x_test, name):
    x_pred = model.predict(x_test)
    df_test['healthy'] = x_pred[:,0]
    df_test['multiple_diseases'] = x_pred[:,1]
    df_test['rust'] = x_pred[:,2]
    df_test['scab'] = x_pred[:,3]
    df_test.to_csv(name, index = None)

In [6]:
#See distribution of samples
y_train.sum(axis = 0)

array([2051,  357, 2508, 2368], dtype=int64)

# First approach: Batch Confusion Norm

In [8]:
def model_plot(history):

    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['validation_categorical_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'validation'], loc='upper left')
    plt.show()

In [9]:
def print_score(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [15]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [10]:
def ASSP(output, act_name): # TODO: Check if it is well built. Not like in paper. Maybe different image size?
    ASSP = Conv2D(filters = 2048, kernel_size = (1,1))(output)
    ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 2)(ASSP)
#     ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 4)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 6)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 7)(ASSP)

    ASSP = Activation(act_name)(ASSP)
    
    return ASSP

In [11]:


def compute_bcn_loss(y_pred):

    
    matrix = K.dot(K.transpose(y_pred), y_pred)
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    bcn_loss = K.sum(eigenvalues) 
    
    print(eigenvalues)
    print(type(eigenvalues))
    print(type(y_pred))
    print('y_pred shape: ', y_pred.shape)
    print('yPred.transpose shape: ', K.transpose(y_pred).shape)
    print('matrix shape: ', matrix.shape)
    print('eigenvalues shape: ', eigenvalues.shape)
    
    print()
    
    
    
    return bcn_loss



def custom_loss(y_true, y_pred):
    
    categorical_crossentropy = losses.categorical_crossentropy(y_true, y_pred)
    
    BCN_loss = compute_bcn_loss(y_pred)
    
    loss_total = categorical_crossentropy + BCN_loss
    
    return loss_total
    
    
    
    
    

In [18]:
inputs = Input(shape = (224, 224, 3))
out = model_resnet(inputs)
features = ASSP(out, 'relu')
attention = ASSP(out, 'sigmoid')
tensor_total = Multiply()([features, attention])
tensor_total_flat = Flatten()(tensor_total)
tensor_total_flat = Dense(256, activation = 'relu')(tensor_total_flat) # TODO: Check if it should be relu or another
tensor_total_flat = Dense(4, activation = 'softmax')(tensor_total_flat)

model = Model(inputs=inputs, outputs= tensor_total_flat)

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['accuracy'])

print(model.summary())

Tensor("loss/dense_2_loss/custom_loss/SelfAdjointEigV2:0", shape=(4,), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
y_pred shape:  (None, 4)
yPred.transpose shape:  (4, None)
matrix shape:  (4, 4)
eigenvalues shape:  (4,)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 7, 7, 2048)   23587712    input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 7, 7, 2048)   4196352     resnet50[1][0]                   

In [19]:
model.fit(x_train, y_train, batch_size = 2, epochs = 3, validation_data = (x_val, y_val))

Train on 7284 samples, validate on 1821 samples
Epoch 1/3


ResourceExhaustedError:  OOM when allocating tensor with shape[3,3,2048,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node mul_1092 (defined at C:\Programs_julen\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_50206]

Function call stack:
keras_scratch_graph


In [ ]:
# TODO: In order to set the batch size and each batch with different labels, we should use fit_generator or train_on_batch

# Simple model

In [10]:
def build_model(x_train, y_train, batch_size, epochs, verbose = True):
    
    
    ##model building
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu', input_shape = (224, 224, 3)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(1024, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))

    model.add(Dense(4, activation='softmax'))
    

    print(model.summary())
    start_time = time.time()
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['categorical_accuracy'])
    history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), batch_size = batch_size, epochs = epochs, verbose = verbose)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return (model, history)

In [14]:
model_old, history = build_model(x_train, y_train, batch_size = 16, epochs = 1)
model_plot(history)
print_score(model_old, x_test, y_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 54, 54, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 52, 52, 512)      

KeyError: 'acc'

In [18]:
df_test['healthy'] = x_pred[:,0]
df_test['multiple_diseases'] = x_pred[:,1]
df_test['rust'] = x_pred[:,2]
df_test['scab'] = x_pred[:,3]
df_test.to_csv('FGVC_submission_.csv', index = None)

In [17]:
df_test

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.205251,0.045785,0.368202,0.380761
1,Test_1,0.231453,0.047746,0.345878,0.374923
2,Test_2,0.223623,0.071676,0.342834,0.361867
3,Test_3,0.308657,0.102331,0.295816,0.293197
4,Test_4,0.230924,0.044711,0.364794,0.359571
...,...,...,...,...,...
1816,Test_1816,0.220903,0.053204,0.343984,0.381910
1817,Test_1817,0.273419,0.108838,0.302602,0.315141
1818,Test_1818,0.324635,0.099112,0.290823,0.285430
1819,Test_1819,0.337709,0.099663,0.284925,0.277703


# Second approach: Batch Confusion Norm

In [7]:
from keras import backend as K
from keras.layers import Layer, InputSpec
from keras.legacy import interfaces




class GlobalKMaxPooling2D(Layer): #Inherits the properties of _Pooling2D class
    """K Max Pooling operation for spatial data.
    
    # Arguments
        
        data_format: A string,
            one of `"channels_last"` (default) or `"channels_first"`.
            The ordering of the dimensions in the inputs.
            `"channels_last"` corresponds to inputs with shape
            `(batch, height, width, channels)` while `"channels_first"`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be `"channels_last"`.
            
        K: An Integer,
            states the number of selected maximal values over which the
            average is going to be computed.
            
            
    # Input shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, rows, cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, rows, cols)`
            
    # Output shape
    
        - If `data_format='channels_last'`:
            4D tensor with shape:
            `(batch_size, pooled_rows, pooled_cols, channels)`
            
        - If `data_format='channels_first'`:
            4D tensor with shape:
            `(batch_size, channels, pooled_rows, pooled_cols)`    
    
    
    """
    
    #@interfaces.legacy_pooling2d_support
    def __init__(self, data_format=None, k = 10, **kwargs):
        super(GlobalKMaxPooling2D, self).__init__(**kwargs)
        self.data_format = K.normalize_data_format(data_format)
        self.input_spec = InputSpec(ndim=4)
        self.k = k

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            return (input_shape[0], input_shape[3])
        else:
            return (input_shape[0], input_shape[1])

    def get_config(self):
        config = {'data_format': self.data_format, 'k' : self.k}
        base_config = super(GlobalKMaxPooling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
        
    def call(self, inputs):
        if self.data_format == 'channels_last':
            # Here first sort
            # Then take K maximum values
            # Then average them
            k = self.k
#             print(inputs.get_shape())
#             print(type(inputs.get_shape()[0]))
#             print(tf.shape(inputs)[0])
            input_reshaped = tf.reshape(inputs, [tf.shape(inputs)[0], -1, tf.shape(inputs)[3]])
            input_reshaped = tf.reshape(input_reshaped, [tf.shape(input_reshaped)[0], tf.shape(input_reshaped)[2], tf.shape(input_reshaped)[1]])
            top_k = tf.math.top_k(input_reshaped, k=k, sorted = True, name = None)[0]
#             print(type(top_k))
            mean = tf.keras.backend.mean(top_k, axis = 2)
            #assert ((input_reshaped.get_shape()[0], input_reshaped.get_shape()[-1]) == mean.get_shape())
        
        return mean

    
    
    

In [8]:
import keras
from keras import models, Sequential
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Reshape
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50

In [19]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [20]:

model = Sequential()
model.add(model_resnet)
model.add(GlobalKMaxPooling2D(data_format = 'channels_last' , k = 8))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
print(model.summary())


(None, 7, 7, 2048)
<class 'NoneType'>
Tensor("global_k_max_pooling2d_2/strided_slice:0", shape=(), dtype=int32)
<class 'tensorflow.python.framework.ops.Tensor'>
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_k_max_pooling2d_2 (Gl (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               262272    
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 516       
Total params: 23,850,500
Trainable params: 23,797,380
Non-trainable params: 53,120
_________________________________________________________________
None


In [21]:
history = model.fit(x_train, y_train, batch_size = 10, epochs = 3, validation_data = (x_val, y_val))

Train on 7284 samples, validate on 1821 samples
Epoch 1/3
7284/7284 [==============================] - 460s 63ms/step - loss: 0.9012 - categorical_accuracy: 0.6938 - val_loss: 0.9141 - val_categorical_accuracy: 0.7496
Epoch 2/3
7284/7284 [==============================] - 448s 62ms/step - loss: 0.3838 - categorical_accuracy: 0.8870 - val_loss: 0.4687 - val_categorical_accuracy: 0.8622
Epoch 3/3
7284/7284 [==============================] - 447s 61ms/step - loss: 0.3056 - categorical_accuracy: 0.9076 - val_loss: 0.4028 - val_categorical_accuracy: 0.8847


In [15]:
model_plot(history)

NameError: name 'model_plot' is not defined

In [23]:
predict_and_save(model, x_test, name = 'FGVC_submission_2_approach.csv')